In [ ]:
from graph_tool.all import *
import graph_tool.all as gt
import scipy as sp
import numpy as np
from tqdm import tqdm
import os

In [ ]:
Node = 5000

m = 2

times = 10

beta_list = [1,-1]

name_list = ["SF"]

count = 1
recalculate = 5000

In [ ]:
# 第二固有値と固有ベクトルを算出する関数
def no_normalized_laplacian(g):
    L = gt.laplacian(g)
    L_sparse = sp.sparse.csr_matrix(L)
    eigenvalues, eigenvectors = sp.sparse.linalg.eigsh(L_sparse, k=2, which='SM')
    return eigenvalues[1], eigenvectors[:, 1]

def max_abs_fiedler(fiedler_vector, g):
    # フィードラーベクトルを1次元配列として取得
    fiedler_vector = np.array(fiedler_vector)

    # 差分行列の計算（|fiedler[i] - fiedler[j]|）
    diff_matrix = np.abs(fiedler_vector[:, None] - fiedler_vector[None, :])

    # 隣接行列を取得
    adjacency = gt.adjacency(g).toarray()

    # エッジがないノードペアのみを対象とするフィルタ
    mask = (adjacency == 0)  # 隣接していないノードペアが True

    # 差分行列にマスクを適用
    diff_matrix[~mask] = -np.inf  # エッジが存在する部分は無効化 (-∞)

    # 最大値とそのインデックスを取得
    max_diff = np.max(diff_matrix)
    node_pair = np.unravel_index(np.argmax(diff_matrix), diff_matrix.shape)

    np_tuple = (node_pair)

    # 通常の Python の整数タプルに変換
    python_tuple = tuple(int(x) for x in np_tuple)

    return python_tuple

def min_abs_fiedler(fiedler_vector, g, removed_edges):
    # フィードラーベクトルを1次元配列として取得
    fiedler_vector = np.array(fiedler_vector)

    # グラフのエッジリストを取得（削除されたエッジは除外）
    edges = [(e.source(), e.target()) for e in g.edges() if e not in removed_edges]

    # エッジリストからノードペアのインデックスを抽出
    node_pairs = np.array(edges, dtype=int)  # データ型を明示的に整数型に変換

    # 各エッジに対応するノードペアの差分を計算
    diffs = np.abs(fiedler_vector[node_pairs[:, 0]] - fiedler_vector[node_pairs[:, 1]])

    # 最小差分と対応するノードペアを取得
    min_diff_idx = np.argmin(diffs)
    min_diff = diffs[min_diff_idx]
    node_pair = tuple(node_pairs[min_diff_idx])

    np_tuple = (node_pair)

    # 通常の Python の整数タプルに変換
    python_tuple = tuple(int(x) for x in np_tuple)

    return python_tuple

# 最大連結成分を求める
def max_component(g):
    l = gt.label_largest_component(g)
    u = gt.GraphView(g, vfilt=l)
    sum_u = u.num_vertices()
    return sum_u

In [ ]:
SF_μ_List = []
ER_μ_List = []
for net in name_list:
    if "SF" in net:
        SF_λ_list = []
    else:
        ER_λ_list = []

    for i in range(times):
        g = load_graph(f"~/o_t_hayashilab/Network_data/graph-tool/pure_pk/N={Node}/{net}/{i}.gt.gz")
        # 初期グラフを保存する前にコピーする
        G = g.copy()

        if "SF" in net:
            SF_μ_list = []
        else:
            ER_μ_list = []

        μ, fiedler_vector = no_normalized_laplacian(G)

        if "SF" in net:
            SF_μ_list.append(μ)
        else:
            ER_μ_list.append(μ)

        for _ in tqdm(range(recalculate)):

            not_node_pair = max_abs_fiedler(fiedler_vector, G)

            G.add_edge(G.vertex(not_node_pair[0]), G.vertex(not_node_pair[1]))

            removed_edges = []

            μ, fiedler_vector = no_normalized_laplacian(G)

            node_pair = min_abs_fiedler(fiedler_vector, G, removed_edges)

            attempts = 0
            max_attempts = 100
            while attempts < max_attempts:
                edge_to_remove = G.edge(G.vertex(node_pair[0]), G.vertex(node_pair[1]))
                if edge_to_remove is None:
                    raise ValueError("Invalid edge descriptor")
                removed_edges.append(edge_to_remove)
                G.remove_edge(edge_to_remove)

                if max_component(G) == Node:
                    break
                else:
                   G.add_edge(G.vertex(node_pair[0]), G.vertex(node_pair[1]))
                   node_pair = min_abs_fiedler(fiedler_vector, G, removed_edges)
                   attempts += 1

            μ, fiedler_vector = no_normalized_laplacian(G)
            if "SF" in net:
                SF_μ_list.append(μ)
            else:
                ER_μ_list.append(μ)

        if "SF" in net:
            SF_λ_list.append(SF_μ_list)
            directory = (f"~/o_t_hayashilab/Network_data/graph-tool/rewire_laplacian/N={Node}/SF/c={count}_re={recalculate}/")
            os.makedirs(os.path.expanduser(directory), exist_ok=True)

            filename = directory + f"{i}.gt.gz"
            # ネットワークを保存
            G.save(filename)
        else:
            ER_λ_list.append(ER_μ_list)
            directory = (f"~/o_t_hayashilab/Network_data/graph-tool/rewire_laplacian/N={Node}/ER/c={count}_re={recalculate}/")
            os.makedirs(os.path.expanduser(directory), exist_ok=True)

            filename = directory + f"{i}.gt.gz"
            # ネットワークを保存
            G.save(filename)

    if "SF" in net:
        SF = np.mean(SF_λ_list, axis=0)
        SF_μ_List.append(SF)
    else:
        ER = np.mean(ER_λ_list, axis=0)
        ER_μ_List.append(ER)

In [ ]:
# 保存ディレクトリのパスを指定
sf_save_dir = f"~/o_t_hayashilab/Network_information/rewire_laplacian/iteration_μ/N={Node}/SF"
#er_save_dir = f"~/o_t_hayashilab/Network_information/rewire_laplacian/iteration_μ/N={Node}/ER"

# ディレクトリが存在しない場合は作成
os.makedirs(os.path.expanduser(sf_save_dir), exist_ok=True)
#os.makedirs(os.path.expanduser(er_save_dir), exist_ok=True)

# 保存ファイル名
sf_save_path = os.path.join(sf_save_dir, f"c=1_re=5000_0-9.npy")
#er_save_path = os.path.join(er_save_dir, f"c=1_re=5000_10-19.npy")

# 1次元配列を2次元配列に変換（1行に並べる形にする）
SF_μ_2D = np.reshape(SF_μ_List[0], (1, -1))
#ER_μ_2D = np.reshape(ER_μ_List[0], (1, -1))

# SF_μ_ListとER_μ_Listをnumpy配列に変換して保存
np.save(os.path.expanduser(sf_save_path), SF_μ_2D)
#np.save(os.path.expanduser(er_save_path), ER_μ_2D)